# Tema 5 - Modele de regresie

In [1]:
import numpy as np
import pandas as pd
import sklearn

print('At the moment the program was made:')
print('\tNumPy Version = 1.20.2')
print('\tScykit_learn Version = 0.24.1')
print('\tPandas Version = 1.2.3\n')

print(f'Current NumPy Version = {np.__version__}')
print(f'Current Sklearn Version = {sklearn.__version__}')
print(f'Current Pandas Version = {pd.__version__}')

At the moment the program was made:
	NumPy Version = 1.20.2
	Scykit_learn Version = 0.24.1
	Pandas Version = 1.2.3

Current NumPy Version = 1.20.1
Current Sklearn Version = 0.24.1
Current Pandas Version = 1.2.4


In [2]:
zeros = np.zeros(10)
CPU_model_df = pd.DataFrame({
    'Model':list(zeros), 
    'Search_strategy': list(zeros), 
    'test_neg_mean_absolute_error': list(zeros), 
    'test_neg_mean_squared_error': list(zeros), 
    'test_neg_median_absolute_error': list(zeros),
    'train_neg_mean_absolute_error': list(zeros),
    'train_neg_mean_squared_error': list(zeros),
    'train_neg_median_absolute_error': list(zeros),
    'fit_time': list(zeros),
    'score_time': list(zeros)
})

## CPU Computer Hardware

* CPU Computer Hardware; excludeti din dataset coloanele: vendor name, model name, estimated relative performance; se va estima coloana "published relative performance".

In [3]:
CPU_data = pd.read_csv("data/machine.data", delimiter=",", header=None)
CPU_data = CPU_data.drop([0, 1, 9], axis = 1)
#CPU_data = CPU_data.drop(1, axis = 1)
#CPU_data = CPU_data.drop(9, axis = 1)
CPU_data.head()

,2,3,4,5,6,7,8
0,125,256,6000,256,16,128,198
1,29,8000,32000,32,8,32,269
2,29,8000,32000,32,8,32,220
3,29,8000,32000,32,8,32,172
4,29,8000,16000,32,8,16,132


In [4]:
CPU_X = CPU_data.iloc[:, :-1].values
CPU_y = CPU_data.iloc[:, -1].values

In [5]:
def populate_dataFrame(dataFrame, scores, regression_model, grid_search, index):
    dataFrame.iloc[index, 0] = regression_model
    dataFrame.iloc[index, 1] = grid_search
    dataFrame.iloc[index, 2] = scores['test_neg_mean_absolute_error'].mean()
    dataFrame.iloc[index, 3] = scores['test_neg_mean_squared_error'].mean()
    dataFrame.iloc[index, 4] = scores['test_neg_median_absolute_error'].mean()
    dataFrame.iloc[index, 5] = scores['train_neg_mean_absolute_error'].mean()
    dataFrame.iloc[index, 6] = scores['train_neg_mean_squared_error'].mean()
    dataFrame.iloc[index, 7] = scores['train_neg_median_absolute_error'].mean()
    dataFrame.iloc[index, 8] = scores['fit_time'].mean()
    dataFrame.iloc[index, 9] = scores['score_time'].mean()


### LinearRegression

In [6]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, GridSearchCV, RandomizedSearchCV
from sklearn import metrics

In [7]:
regression = LinearRegression()
parameter_grid = {"fit_intercept":[True, False], "normalize":[True, False], "n_jobs":[None, 2, 4, 6]}

In [8]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(CPU_X, CPU_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'fit_intercept': True, 'n_jobs': None, 'normalize': True}


In [9]:
tunned_regression = LinearRegression(**best_parameters)
grid_scores = cross_validate(tunned_regression, CPU_X, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, grid_scores, 'LinearRegression', 'GridSearchCV', 0)

In [10]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(CPU_X, CPU_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'normalize': True, 'n_jobs': 2, 'fit_intercept': True}


In [11]:
tunned_regression = LinearRegression(**best_parameters)
random_scores = cross_validate(tunned_regression, CPU_X, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, random_scores, 'LinearRegression', 'RandomizedSearchCV', 1)
CPU_model_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.000597,0.000798
1,LinearRegression,RandomizedSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.000608,0.000599
2,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### DecisionTreeRegressor

In [12]:
from sklearn import tree

In [13]:
regression = tree.DecisionTreeRegressor()
parameter_grid = {'splitter':['best', 'random'], 'max_depth': [1, 3, 5, 7], 'min_samples_leaf':[1, 2, 3, 4, 5],
                 'min_weight_fraction_leaf':[0.1, 0.2, 0.3, 0.4, 0.5], 'max_features':['auto', 'log2', 'sqrt', None],
                 'max_leaf_nodes':[None, 10, 20, 30, 40, 50]}

In [14]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(CPU_X, CPU_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'max_depth': 7, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0.1, 'splitter': 'random'}


In [15]:
tuned_regression = tree.DecisionTreeRegressor(**best_parameters)
grid_scores = cross_validate(tuned_regression, CPU_X, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, grid_scores, 'DecisionTreeRegressor', 'GridSearchCV', 2)

In [16]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(CPU_X, CPU_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'splitter': 'random', 'min_weight_fraction_leaf': 0.1, 'min_samples_leaf': 3, 'max_leaf_nodes': 10, 'max_features': 'auto', 'max_depth': 3}


In [17]:
tuned_regression = tree.DecisionTreeRegressor(**best_parameters)
random_scores = cross_validate(tuned_regression, CPU_X, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, random_scores, 'DecisionTreeRegressor', 'RandomizedSearchCV', 3)

CPU_model_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.000597,0.000798
1,LinearRegression,RandomizedSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.000608,0.000599
2,DecisionTreeRegressor,GridSearchCV,-79.213846,-17520.960833,-46.667866,-68.021999,-18079.578696,-33.136773,0.000199,0.000716
3,DecisionTreeRegressor,RandomizedSearchCV,-53.860236,-14777.125400,-20.767307,-55.005050,-13354.291819,-21.005613,0.000598,0.000800
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### KernelRidge

In [18]:
from sklearn.kernel_ridge import KernelRidge

In [19]:
regression = KernelRidge()
parameter_grid = {'kernel':['linear', 'rbf'], 'alpha':[1, 2, 3, 5], 'gamma':[None, 0.1, 0.001]}

In [20]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(CPU_X, CPU_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'alpha': 5, 'gamma': None, 'kernel': 'linear'}


In [21]:
tuned_regression = KernelRidge(**best_parameters)
grid_scores = cross_validate(tuned_regression, CPU_X, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, grid_scores, 'KernelRidge', 'GridSearchCV', 4)

In [22]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(CPU_X, CPU_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'kernel': 'linear', 'gamma': 0.1, 'alpha': 5}


In [23]:
tuned_regression = KernelRidge(**best_parameters)
random_scores = cross_validate(tuned_regression, CPU_X, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, random_scores, 'KernelRidge', 'RandomizedSearchCV', 5)

CPU_model_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.000597,0.000798
1,LinearRegression,RandomizedSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.000608,0.000599
2,DecisionTreeRegressor,GridSearchCV,-79.213846,-17520.960833,-46.667866,-68.021999,-18079.578696,-33.136773,0.000199,0.000716
3,DecisionTreeRegressor,RandomizedSearchCV,-53.860236,-14777.125400,-20.767307,-55.005050,-13354.291819,-21.005613,0.000598,0.000800
4,KernelRidge,GridSearchCV,-47.222356,-8067.968394,-30.918492,-39.472752,-3987.726525,-24.955859,0.001616,0.001204
5,KernelRidge,RandomizedSearchCV,-47.222356,-8067.968394,-30.918492,-39.472752,-3987.726525,-24.955859,0.003192,0.002195
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### GaussianProcessRegressor

In [24]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, DotProduct

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
CPU_X_scaled = scaler.fit_transform(CPU_X)
print(CPU_X_scaled)

[[0.07282535 0.00601202 0.09284284 1.         0.30769231 0.72727273]
 [0.00809171 0.24849699 0.4994995  0.125      0.15384615 0.18181818]
 [0.00809171 0.24849699 0.4994995  0.125      0.15384615 0.18181818]
 ...
 [0.07282535 0.06062124 0.12412412 0.         0.03846154 0.07954545]
 [0.31220499 0.01402806 0.12412412 0.125      0.         0.        ]
 [0.31220499 0.02930862 0.06156156 0.         0.         0.        ]]


In [26]:
regression = GaussianProcessRegressor()
parameter_grid = [{"alpha":[1e-2, 1e-3], "kernel":[RBF(l) for l in np.logspace(-1, 1, 2)]},
                 {"alpha":[1e-2, 1e-3], "kernel": [DotProduct(sigma_0) for sigma_0 in np.logspace(-1, 1, 2)]}]

In [27]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(CPU_X_scaled, CPU_y)
best_parameters = grid_search.best_params_
print(best_parameters)

D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn

D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better val

{'alpha': 0.01, 'kernel': DotProduct(sigma_0=0.1)}


D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


In [28]:
tuned_regression = GaussianProcessRegressor(**best_parameters)
grid_scores = cross_validate(tuned_regression, CPU_X_scaled, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, grid_scores, 'GaussianProcessRegressor', 'GridSearchCV', 6)

D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The 

In [29]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(CPU_X_scaled, CPU_y)
best_parameters = random_search.best_params_
print(best_parameters)

D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\model_selection\_search.py:285: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:402: ConvergenceWarning: The optimal value found for dimension 0 of parameter length_scale is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\skle

D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\_gpr.py:509: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessi

{'kernel': DotProduct(sigma_0=0.1), 'alpha': 0.01}


D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "


In [30]:
tuned_regression = GaussianProcessRegressor(**best_parameters)
random_scores = cross_validate(tuned_regression, CPU_X_scaled, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, random_scores, 'GaussianProcessRegressor', 'RandomizedSearchCV', 7)

CPU_model_df.head(n=10)

D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The optimal value found for "
D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\gaussian_process\kernels.py:411: ConvergenceWarning: The optimal value found for dimension 0 of parameter sigma_0 is close to the specified upper bound 100000.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn("The 

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.000597,0.000798
1,LinearRegression,RandomizedSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.000608,0.000599
2,DecisionTreeRegressor,GridSearchCV,-79.213846,-17520.960833,-46.667866,-68.021999,-18079.578696,-33.136773,0.000199,0.000716
3,DecisionTreeRegressor,RandomizedSearchCV,-53.860236,-14777.125400,-20.767307,-55.005050,-13354.291819,-21.005613,0.000598,0.000800
4,KernelRidge,GridSearchCV,-47.222356,-8067.968394,-30.918492,-39.472752,-3987.726525,-24.955859,0.001616,0.001204
5,KernelRidge,RandomizedSearchCV,-47.222356,-8067.968394,-30.918492,-39.472752,-3987.726525,-24.955859,0.003192,0.002195
6,GaussianProcessRegressor,GridSearchCV,-43.320208,-6366.077037,-27.112500,-36.664766,-3243.312565,-25.520312,0.017181,0.000605
7,GaussianProcessRegressor,RandomizedSearchCV,-43.320208,-6366.077037,-27.112500,-36.664766,-3243.312565,-25.520312,0.015067,0.000994
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### LogisticRegression

In [31]:
from sklearn.linear_model import LogisticRegression

In [32]:
regression = LogisticRegression()
parameter_grid = {'penalty': ['l1', 'l2'], 'C': np.logspace(-4, 4, 20), 'solver':['liblinear'], 'max_iter':[1e4]}

In [33]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(CPU_X_scaled, CPU_y)
best_parameters = grid_search.best_params_
print(best_parameters)

D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


{'C': 1438.44988828766, 'max_iter': 10000.0, 'penalty': 'l2', 'solver': 'liblinear'}


In [34]:
tuned_regression = LogisticRegression(**best_parameters)
grid_scores = cross_validate(tuned_regression, CPU_X_scaled, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, grid_scores, 'LogisticRegression', 'GridSearchCV', 8)

D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [35]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(CPU_X_scaled, CPU_y)
best_parameters = random_search.best_params_
print(best_parameters)

D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


{'solver': 'liblinear', 'penalty': 'l1', 'max_iter': 10000.0, 'C': 545.5594781168514}


In [36]:
tuned_regression = LogisticRegression(**best_parameters)
random_scores = cross_validate(tuned_regression, CPU_X_scaled, CPU_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(CPU_model_df, random_scores, 'LogisticRegression', 'RandomizedSearchCV', 9)

CPU_model_df.head(n=10)

D:\Programe\AnacondaNavigator\envs\IDS\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.000597,0.000798
1,LinearRegression,RandomizedSearchCV,-43.378262,-6383.656697,-27.053792,-36.695674,-3243.698611,-25.581576,0.000608,0.000599
2,DecisionTreeRegressor,GridSearchCV,-79.213846,-17520.960833,-46.667866,-68.021999,-18079.578696,-33.136773,0.000199,0.000716
3,DecisionTreeRegressor,RandomizedSearchCV,-53.860236,-14777.125400,-20.767307,-55.005050,-13354.291819,-21.005613,0.000598,0.000800
4,KernelRidge,GridSearchCV,-47.222356,-8067.968394,-30.918492,-39.472752,-3987.726525,-24.955859,0.001616,0.001204
5,KernelRidge,RandomizedSearchCV,-47.222356,-8067.968394,-30.918492,-39.472752,-3987.726525,-24.955859,0.003192,0.002195
6,GaussianProcessRegressor,GridSearchCV,-43.320208,-6366.077037,-27.112500,-36.664766,-3243.312565,-25.520312,0.017181,0.000605
7,GaussianProcessRegressor,RandomizedSearchCV,-43.320208,-6366.077037,-27.112500,-36.664766,-3243.312565,-25.520312,0.015067,0.000994
8,LogisticRegression,GridSearchCV,-47.049245,-8983.801394,-15.800000,-12.373460,-659.393449,-3.600000,0.030623,0.001000
9,LogisticRegression,RandomizedSearchCV,-47.334495,-11995.242509,-17.600000,-9.928735,-668.951718,-0.200000,0.224331,0.000399


In [37]:
CPU_model_df.iloc[:, 2:] = np.abs(CPU_model_df.iloc[:, 2:])
CPU_model_df = CPU_model_df.rename(columns={'test_neg_mean_absolute_error': 'test_mean_absolute_error', 'test_neg_mean_squared_error': 'test_mean_squared_error', 
                                            'test_neg_median_absolute_error': 'test_median_absolute_error', 'train_neg_mean_absolute_error': 'train_mean_absolute_error', 
                                            'train_neg_mean_squared_error': 'train_mean_squared_error', 'train_neg_median_absolute_error': 'train_median_absolute_error'})
CPU_model_df.head(n=10)

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,43.378262,6383.656697,27.053792,36.695674,3243.698611,25.581576,0.000597,0.000798
1,LinearRegression,RandomizedSearchCV,43.378262,6383.656697,27.053792,36.695674,3243.698611,25.581576,0.000608,0.000599
2,DecisionTreeRegressor,GridSearchCV,79.213846,17520.960833,46.667866,68.021999,18079.578696,33.136773,0.000199,0.000716
3,DecisionTreeRegressor,RandomizedSearchCV,53.860236,14777.125400,20.767307,55.005050,13354.291819,21.005613,0.000598,0.000800
4,KernelRidge,GridSearchCV,47.222356,8067.968394,30.918492,39.472752,3987.726525,24.955859,0.001616,0.001204
5,KernelRidge,RandomizedSearchCV,47.222356,8067.968394,30.918492,39.472752,3987.726525,24.955859,0.003192,0.002195
6,GaussianProcessRegressor,GridSearchCV,43.320208,6366.077037,27.112500,36.664766,3243.312565,25.520312,0.017181,0.000605
7,GaussianProcessRegressor,RandomizedSearchCV,43.320208,6366.077037,27.112500,36.664766,3243.312565,25.520312,0.015067,0.000994
8,LogisticRegression,GridSearchCV,47.049245,8983.801394,15.800000,12.373460,659.393449,3.600000,0.030623,0.001000
9,LogisticRegression,RandomizedSearchCV,47.334495,11995.242509,17.600000,9.928735,668.951718,0.200000,0.224331,0.000399


In [38]:
styler = CPU_model_df.head(10).style.highlight_max(color='tomato').highlight_min(color='lightgreen')
styler

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,43.378262,6383.656697,27.053792,36.695674,3243.698611,25.581576,0.000597,0.000798
1,LinearRegression,RandomizedSearchCV,43.378262,6383.656697,27.053792,36.695674,3243.698611,25.581576,0.000608,0.000599
2,DecisionTreeRegressor,GridSearchCV,79.213846,17520.960833,46.667866,68.021999,18079.578696,33.136773,0.000199,0.000716
3,DecisionTreeRegressor,RandomizedSearchCV,53.860236,14777.125400,20.767307,55.005050,13354.291819,21.005613,0.000598,0.000800
4,KernelRidge,GridSearchCV,47.222356,8067.968394,30.918492,39.472752,3987.726525,24.955859,0.001616,0.001204
5,KernelRidge,RandomizedSearchCV,47.222356,8067.968394,30.918492,39.472752,3987.726525,24.955859,0.003192,0.002195
6,GaussianProcessRegressor,GridSearchCV,43.320208,6366.077037,27.112500,36.664766,3243.312565,25.520312,0.017181,0.000605
7,GaussianProcessRegressor,RandomizedSearchCV,43.320208,6366.077037,27.112500,36.664766,3243.312565,25.520312,0.015067,0.000994
8,LogisticRegression,GridSearchCV,47.049245,8983.801394,15.800000,12.373460,659.393449,3.600000,0.030623,0.001000
9,LogisticRegression,RandomizedSearchCV,47.334495,11995.242509,17.600000,9.928735,668.951718,0.200000,0.224331,0.000399


In [39]:
html = styler.render()
file = open("CPU_Computer_Hardware_DataSet.html", "w")
file.write(html)
file.close()

## Boston Housing

In [40]:
zeros = np.zeros(10)
BostonHousing_df = pd.DataFrame({
    'Model':list(zeros), 
    'Search_strategy': list(zeros), 
    'test_neg_mean_absolute_error': list(zeros), 
    'test_neg_mean_squared_error': list(zeros), 
    'test_neg_median_absolute_error': list(zeros),
    'train_neg_mean_absolute_error': list(zeros),
    'train_neg_mean_squared_error': list(zeros),
    'train_neg_median_absolute_error': list(zeros),
    'fit_time': list(zeros),
    'score_time': list(zeros)
})
BostonHousing_df.head()

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
BH_data = pd.read_csv("data/housing.data", delimiter = r'\s+', header = None)
BH_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2


In [42]:
housing_X = BH_data.iloc[:, :-1].values
housing_y = BH_data.iloc[:, -1].values

### SupportVectorRegression

In [43]:
from sklearn.svm import SVR

In [44]:
regression = SVR(epsilon=0.01)
parameter_grid = [{'kernel': ['rbf'], 'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5, 0.6, 0.9],'C': [1, 10, 100, 1000, 10000]}]

In [45]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_X, housing_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}


In [46]:
tuned_regression = SVR(**best_parameters)
grid_scores = cross_validate(tuned_regression, housing_X, housing_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(BostonHousing_df, grid_scores, 'SupportVectorRegression', 'GridSearchCV', 0)

In [47]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(housing_X, housing_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'kernel': 'rbf', 'gamma': 0.0001, 'C': 100}


In [48]:
tuned_regression = SVR(**best_parameters)
random_scores = cross_validate(tuned_regression, housing_X, housing_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(BostonHousing_df, random_scores, 'SupportVectorRegression', 'RandomizedSearchCV', 1)

BostonHousing_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,-5.312487,-60.626008,-3.509054,-2.564362,-22.106605,-1.344217,0.015963,0.004988
1,SupportVectorRegression,RandomizedSearchCV,-5.312487,-60.626008,-3.509054,-2.564362,-22.106605,-1.344217,0.019757,0.005584
2,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### LassoLars

In [49]:
from sklearn import linear_model

In [50]:
regression = linear_model.LassoLars()
parameter_grid = [{'alpha':[0.02, 0.024, 0.015, 0.025, 0.026, 0.03, 0.023, 0.017, 0.033, 0.014, 0.019]}]

In [51]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_X, housing_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'alpha': 0.014}


In [52]:
tuned_regression = linear_model.LassoLars(**best_parameters)
grid_scores = cross_validate(tuned_regression, housing_X, housing_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(BostonHousing_df, grid_scores, 'LassoLars', 'GridSearchCV', 2)

In [53]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(housing_X, housing_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'alpha': 0.014}


In [54]:
tuned_regression = linear_model.LassoLars(**best_parameters)
random_scores = cross_validate(tuned_regression, housing_X, housing_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(BostonHousing_df, random_scores, 'LassoLars', 'RandomizedSearchCV', 3)

BostonHousing_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,-5.312487,-60.626008,-3.509054,-2.564362,-22.106605,-1.344217,0.015963,0.004988
1,SupportVectorRegression,RandomizedSearchCV,-5.312487,-60.626008,-3.509054,-2.564362,-22.106605,-1.344217,0.019757,0.005584
2,LassoLars,GridSearchCV,-4.071615,-36.322984,-2.922391,-3.300380,-23.026079,-2.353271,0.002394,0.000440
3,LassoLars,RandomizedSearchCV,-4.071615,-36.322984,-2.922391,-3.300380,-23.026079,-2.353271,0.003192,0.000792
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### ARDRegression

In [55]:
regression = linear_model.ARDRegression()
parameter_grid = [{'tol':[1e-3, 1e-4], 'alpha_1':[1e-6, 1e-5, 1e-7], 'alpha_2':[1e-6, 1e-5, 1e-7]}]

In [56]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_X, housing_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'alpha_1': 1e-07, 'alpha_2': 1e-05, 'tol': 0.0001}


In [57]:
tuned_regression = linear_model.ARDRegression(**best_parameters)
grid_scores = cross_validate(tuned_regression, housing_X, housing_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(BostonHousing_df, grid_scores, 'ARDRegression', 'GridSearchCV', 4)

In [58]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(housing_X, housing_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'tol': 0.0001, 'alpha_2': 1e-06, 'alpha_1': 1e-07}


In [59]:
tuned_regression = linear_model.ARDRegression(**best_parameters)
random_scores = cross_validate(tuned_regression, housing_X, housing_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(BostonHousing_df, random_scores, 'ARDRegression', 'RandomizedSearchCV', 5)

BostonHousing_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,-5.312487,-60.626008,-3.509054,-2.564362,-22.106605,-1.344217,0.015963,0.004988
1,SupportVectorRegression,RandomizedSearchCV,-5.312487,-60.626008,-3.509054,-2.564362,-22.106605,-1.344217,0.019757,0.005584
2,LassoLars,GridSearchCV,-4.071615,-36.322984,-2.922391,-3.300380,-23.026079,-2.353271,0.002394,0.000440
3,LassoLars,RandomizedSearchCV,-4.071615,-36.322984,-2.922391,-3.300380,-23.026079,-2.353271,0.003192,0.000792
4,ARDRegression,GridSearchCV,-4.251395,-37.438832,-3.268956,-3.256953,-21.315617,-2.325255,0.004495,0.000597
5,ARDRegression,RandomizedSearchCV,-4.251395,-37.438832,-3.268956,-3.256953,-21.315617,-2.325255,0.007576,0.001201
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### PassiveAggressiveRegressor

In [60]:
from sklearn.linear_model import PassiveAggressiveRegressor

In [61]:
regression = PassiveAggressiveRegressor()
parameter_grid = [{'C': np.logspace(-4, 4, 20), 'tol':[1e-3, 1e-4]}]

In [62]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_X, housing_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'C': 3792.690190732246, 'tol': 0.001}


In [63]:
tuned_regression = PassiveAggressiveRegressor(**best_parameters)
grid_scores = cross_validate(tuned_regression, housing_X, housing_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(BostonHousing_df, grid_scores, 'PassiveAggressiveRegressor', 'GridSearchCV', 6)

In [64]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(housing_X, housing_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'tol': 0.001, 'C': 0.08858667904100823}


In [65]:
tuned_regression = PassiveAggressiveRegressor(**best_parameters)
random_scores = cross_validate(tuned_regression, housing_X, housing_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(BostonHousing_df, random_scores, 'PassiveAggressiveRegressor', 'RandomizedSearchCV', 7)

BostonHousing_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,-5.312487,-60.626008,-3.509054,-2.564362,-22.106605,-1.344217,0.015963,0.004988
1,SupportVectorRegression,RandomizedSearchCV,-5.312487,-60.626008,-3.509054,-2.564362,-22.106605,-1.344217,0.019757,0.005584
2,LassoLars,GridSearchCV,-4.071615,-36.322984,-2.922391,-3.300380,-23.026079,-2.353271,0.002394,0.000440
3,LassoLars,RandomizedSearchCV,-4.071615,-36.322984,-2.922391,-3.300380,-23.026079,-2.353271,0.003192,0.000792
4,ARDRegression,GridSearchCV,-4.251395,-37.438832,-3.268956,-3.256953,-21.315617,-2.325255,0.004495,0.000597
5,ARDRegression,RandomizedSearchCV,-4.251395,-37.438832,-3.268956,-3.256953,-21.315617,-2.325255,0.007576,0.001201
6,PassiveAggressiveRegressor,GridSearchCV,-8.686309,-140.930886,-7.883492,-7.959943,-103.015943,-6.781260,0.001197,0.000601
7,PassiveAggressiveRegressor,RandomizedSearchCV,-8.978831,-134.721803,-7.167641,-6.333263,-73.194457,-4.889454,0.001191,0.000399
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### TheilSenRegressor

In [66]:
from sklearn.linear_model import TheilSenRegressor

In [67]:
regression = TheilSenRegressor()
parameter_grid = [{'tol':[1e-3, 1e-4], 'n_jobs':[None, 1, 2, 3, 4]}]

In [68]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(housing_X, housing_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'n_jobs': None, 'tol': 0.001}


In [69]:
tuned_regression = TheilSenRegressor(**best_parameters)
grid_scores = cross_validate(tuned_regression, housing_X, housing_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(BostonHousing_df, grid_scores, 'TheilSenRegressor', 'GridSearchCV', 8)

In [70]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(housing_X, housing_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'tol': 0.001, 'n_jobs': 2}


In [71]:
tuned_regression = TheilSenRegressor(**best_parameters)
random_scores = cross_validate(tuned_regression, housing_X, housing_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(BostonHousing_df, random_scores, 'TheilSenRegressor', 'RandomizedSearchCV', 9)

BostonHousing_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,-5.312487,-60.626008,-3.509054,-2.564362,-22.106605,-1.344217,0.015963,0.004988
1,SupportVectorRegression,RandomizedSearchCV,-5.312487,-60.626008,-3.509054,-2.564362,-22.106605,-1.344217,0.019757,0.005584
2,LassoLars,GridSearchCV,-4.071615,-36.322984,-2.922391,-3.300380,-23.026079,-2.353271,0.002394,0.000440
3,LassoLars,RandomizedSearchCV,-4.071615,-36.322984,-2.922391,-3.300380,-23.026079,-2.353271,0.003192,0.000792
4,ARDRegression,GridSearchCV,-4.251395,-37.438832,-3.268956,-3.256953,-21.315617,-2.325255,0.004495,0.000597
5,ARDRegression,RandomizedSearchCV,-4.251395,-37.438832,-3.268956,-3.256953,-21.315617,-2.325255,0.007576,0.001201
6,PassiveAggressiveRegressor,GridSearchCV,-8.686309,-140.930886,-7.883492,-7.959943,-103.015943,-6.781260,0.001197,0.000601
7,PassiveAggressiveRegressor,RandomizedSearchCV,-8.978831,-134.721803,-7.167641,-6.333263,-73.194457,-4.889454,0.001191,0.000399
8,TheilSenRegressor,GridSearchCV,-4.313369,-42.274579,-2.992114,-3.190446,-24.856322,-2.063490,0.911028,0.000406
9,TheilSenRegressor,RandomizedSearchCV,-4.596601,-50.278065,-3.068775,-3.238774,-26.196090,-2.067000,0.635962,0.000399


In [72]:
BostonHousing_df.iloc[:, 2:] = np.abs(BostonHousing_df.iloc[:, 2:])
BostonHousing_df = BostonHousing_df.rename(columns={'test_neg_mean_absolute_error': 'test_mean_absolute_error', 'test_neg_mean_squared_error': 'test_mean_squared_error', 
                                            'test_neg_median_absolute_error': 'test_median_absolute_error', 'train_neg_mean_absolute_error': 'train_mean_absolute_error', 
                                            'train_neg_mean_squared_error': 'train_mean_squared_error', 'train_neg_median_absolute_error': 'train_median_absolute_error'})
BostonHousing_df.head(n=10)

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,5.312487,60.626008,3.509054,2.564362,22.106605,1.344217,0.015963,0.004988
1,SupportVectorRegression,RandomizedSearchCV,5.312487,60.626008,3.509054,2.564362,22.106605,1.344217,0.019757,0.005584
2,LassoLars,GridSearchCV,4.071615,36.322984,2.922391,3.300380,23.026079,2.353271,0.002394,0.000440
3,LassoLars,RandomizedSearchCV,4.071615,36.322984,2.922391,3.300380,23.026079,2.353271,0.003192,0.000792
4,ARDRegression,GridSearchCV,4.251395,37.438832,3.268956,3.256953,21.315617,2.325255,0.004495,0.000597
5,ARDRegression,RandomizedSearchCV,4.251395,37.438832,3.268956,3.256953,21.315617,2.325255,0.007576,0.001201
6,PassiveAggressiveRegressor,GridSearchCV,8.686309,140.930886,7.883492,7.959943,103.015943,6.781260,0.001197,0.000601
7,PassiveAggressiveRegressor,RandomizedSearchCV,8.978831,134.721803,7.167641,6.333263,73.194457,4.889454,0.001191,0.000399
8,TheilSenRegressor,GridSearchCV,4.313369,42.274579,2.992114,3.190446,24.856322,2.063490,0.911028,0.000406
9,TheilSenRegressor,RandomizedSearchCV,4.596601,50.278065,3.068775,3.238774,26.196090,2.067000,0.635962,0.000399


In [73]:
styler = BostonHousing_df.head(10).style.highlight_max(color='tomato').highlight_min(color='lightgreen')
styler

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,5.312487,60.626008,3.509054,2.564362,22.106605,1.344217,0.015963,0.004988
1,SupportVectorRegression,RandomizedSearchCV,5.312487,60.626008,3.509054,2.564362,22.106605,1.344217,0.019757,0.005584
2,LassoLars,GridSearchCV,4.071615,36.322984,2.922391,3.300380,23.026079,2.353271,0.002394,0.000440
3,LassoLars,RandomizedSearchCV,4.071615,36.322984,2.922391,3.300380,23.026079,2.353271,0.003192,0.000792
4,ARDRegression,GridSearchCV,4.251395,37.438832,3.268956,3.256953,21.315617,2.325255,0.004495,0.000597
5,ARDRegression,RandomizedSearchCV,4.251395,37.438832,3.268956,3.256953,21.315617,2.325255,0.007576,0.001201
6,PassiveAggressiveRegressor,GridSearchCV,8.686309,140.930886,7.883492,7.959943,103.015943,6.781260,0.001197,0.000601
7,PassiveAggressiveRegressor,RandomizedSearchCV,8.978831,134.721803,7.167641,6.333263,73.194457,4.889454,0.001191,0.000399
8,TheilSenRegressor,GridSearchCV,4.313369,42.274579,2.992114,3.190446,24.856322,2.063490,0.911028,0.000406
9,TheilSenRegressor,RandomizedSearchCV,4.596601,50.278065,3.068775,3.238774,26.196090,2.067000,0.635962,0.000399


In [74]:
html = styler.render()
file = open("BostonHousing_DataSet.html", "w")
file.write(html)
file.close()

## Wisconsin Breast Cancer

In [75]:
zeros = np.zeros(10)
WBC_df = pd.DataFrame({
    'Model':list(zeros), 
    'Search_strategy': list(zeros), 
    'test_neg_mean_absolute_error': list(zeros), 
    'test_neg_mean_squared_error': list(zeros), 
    'test_neg_median_absolute_error': list(zeros),
    'train_neg_mean_absolute_error': list(zeros),
    'train_neg_mean_squared_error': list(zeros),
    'train_neg_median_absolute_error': list(zeros),
    'fit_time': list(zeros),
    'score_time': list(zeros)
})
WBC_df.head()

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [76]:
WBC_data = pd.read_csv("data/r_wpbc.data", delimiter = ",", header = None)
WBC_data.head()
assert not np.isnan(WBC_data.values.sum())
WBC_data.head()

,0,1,2,3,4,5,6,7,8,9,...,23,24,25,26,27,28,29,30,31,32
0,5,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,0.07055,0.1865,...,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,31
1,2,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,0.14710,0.2419,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,61
2,0,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,0.08180,0.2333,...,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,116
3,0,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,0.10520,0.2597,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,123
4,0,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,0.10430,0.1809,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,27


In [77]:
WBC_X = WBC_data.iloc[:, :-1].values
WBC_y = WBC_data.iloc[:, -1].values

### LinearRegression

In [78]:
regression = LinearRegression()
parameter_grid = {"fit_intercept":[True, False], "normalize":[True, False], "n_jobs":[None, 2, 4, 6]}

In [79]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(WBC_X, WBC_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'fit_intercept': False, 'n_jobs': None, 'normalize': True}


In [80]:
tunned_regression = LinearRegression(**best_parameters)
grid_scores = cross_validate(tunned_regression, WBC_X, WBC_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(WBC_df, grid_scores, 'LinearRegression', 'GridSearchCV', 0)

In [81]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(WBC_X, WBC_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'normalize': False, 'n_jobs': 4, 'fit_intercept': False}


In [82]:
tunned_regression = LinearRegression(**best_parameters)
random_scores = cross_validate(tunned_regression, WBC_X, WBC_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(WBC_df, random_scores, 'LinearRegression', 'RandomizedSearchCV', 1)
WBC_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-29.727065,-1339.66409,-28.793519,-22.279819,-745.178079,-19.407585,0.001595,0.001401
1,LinearRegression,RandomizedSearchCV,-29.727065,-1339.66409,-28.793519,-22.279819,-745.178079,-19.407585,0.001397,0.000998
2,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### DecisionTreeRegressor

In [83]:
regression = tree.DecisionTreeRegressor()
parameter_grid = {'splitter':['best', 'random'], 'max_depth': [1, 3, 5, 7], 'min_samples_leaf':[1, 2, 3, 4, 5],
                 'min_weight_fraction_leaf':[0.1, 0.2, 0.3, 0.4, 0.5], 'max_features':['auto', 'log2', 'sqrt', None],
                 'max_leaf_nodes':[None, 10, 20, 30, 40, 50]}

In [84]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(WBC_X, WBC_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'max_depth': 7, 'max_features': 'sqrt', 'max_leaf_nodes': 40, 'min_samples_leaf': 1, 'min_weight_fraction_leaf': 0.3, 'splitter': 'best'}


In [85]:
tuned_regression = tree.DecisionTreeRegressor(**best_parameters)
grid_scores = cross_validate(tuned_regression, WBC_X, WBC_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(WBC_df, grid_scores, 'DecisionTreeRegressor', 'GridSearchCV', 2)

In [86]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(WBC_X, WBC_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'splitter': 'best', 'min_weight_fraction_leaf': 0.2, 'min_samples_leaf': 4, 'max_leaf_nodes': 40, 'max_features': None, 'max_depth': 5}


In [87]:
tunned_regression = tree.DecisionTreeRegressor(**best_parameters)
random_scores = cross_validate(tunned_regression, WBC_X, WBC_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(WBC_df, random_scores, 'DecisionTreeRegressor', 'RandomizedSearchCV', 3)
WBC_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-29.727065,-1339.664090,-28.793519,-22.279819,-745.178079,-19.407585,0.001595,0.001401
1,LinearRegression,RandomizedSearchCV,-29.727065,-1339.664090,-28.793519,-22.279819,-745.178079,-19.407585,0.001397,0.000998
2,DecisionTreeRegressor,GridSearchCV,-30.358121,-1255.049823,-27.908023,-26.695304,-1007.952412,-25.005957,0.000790,0.000604
3,DecisionTreeRegressor,RandomizedSearchCV,-30.672376,-1289.762094,-27.047798,-25.208542,-919.618592,-22.573773,0.001190,0.000793
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### KernelRidge

In [88]:
regression = KernelRidge()
parameter_grid = {'kernel':['linear', 'rbf'], 'alpha':[1, 2, 3, 5], 'gamma':[None, 0.1, 0.001]}

In [89]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(WBC_X, WBC_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'alpha': 5, 'gamma': None, 'kernel': 'linear'}


In [90]:
tuned_regression = KernelRidge(**best_parameters)
grid_scores = cross_validate(tuned_regression, WBC_X, WBC_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(WBC_df, grid_scores, 'KernelRidge', 'GridSearchCV', 4)

In [91]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(WBC_X, WBC_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'kernel': 'linear', 'gamma': 0.001, 'alpha': 5}


In [92]:
tunned_regression = KernelRidge(**best_parameters)
random_scores = cross_validate(tunned_regression, WBC_X, WBC_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(WBC_df, random_scores, 'KernelRidge', 'RandomizedSearchCV', 5)
WBC_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-29.727065,-1339.664090,-28.793519,-22.279819,-745.178079,-19.407585,0.001595,0.001401
1,LinearRegression,RandomizedSearchCV,-29.727065,-1339.664090,-28.793519,-22.279819,-745.178079,-19.407585,0.001397,0.000998
2,DecisionTreeRegressor,GridSearchCV,-30.358121,-1255.049823,-27.908023,-26.695304,-1007.952412,-25.005957,0.000790,0.000604
3,DecisionTreeRegressor,RandomizedSearchCV,-30.672376,-1289.762094,-27.047798,-25.208542,-919.618592,-22.573773,0.001190,0.000793
4,KernelRidge,GridSearchCV,-29.579619,-1253.368321,-29.167524,-24.902682,-907.346397,-22.537228,0.002187,0.001110
5,KernelRidge,RandomizedSearchCV,-29.579619,-1253.368321,-29.167524,-24.902682,-907.346397,-22.537228,0.002391,0.001207
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### SupportVectorRegression

In [93]:
regression = SVR(epsilon=0.01)
parameter_grid = [{'kernel': ['rbf'], 'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5, 0.6, 0.9],'C': [1, 10, 100, 1000, 10000]}]

In [94]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(WBC_X, WBC_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}


In [95]:
tuned_regression = SVR(**best_parameters)
grid_scores = cross_validate(tuned_regression, WBC_X, WBC_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(WBC_df, grid_scores, 'SupportVectorRegression', 'GridSearchCV', 6)

In [96]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(WBC_X, WBC_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'kernel': 'rbf', 'gamma': 0.6, 'C': 1000}


In [97]:
tunned_regression = SVR(**best_parameters)
random_scores = cross_validate(tunned_regression, WBC_X, WBC_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(WBC_df, random_scores, 'SupportVectorRegression', 'RandomizedSearchCV', 7)
WBC_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-29.727065,-1339.664090,-28.793519,-22.279819,-745.178079,-19.407585,0.001595,0.001401
1,LinearRegression,RandomizedSearchCV,-29.727065,-1339.664090,-28.793519,-22.279819,-745.178079,-19.407585,0.001397,0.000998
2,DecisionTreeRegressor,GridSearchCV,-30.358121,-1255.049823,-27.908023,-26.695304,-1007.952412,-25.005957,0.000790,0.000604
3,DecisionTreeRegressor,RandomizedSearchCV,-30.672376,-1289.762094,-27.047798,-25.208542,-919.618592,-22.573773,0.001190,0.000793
4,KernelRidge,GridSearchCV,-29.579619,-1253.368321,-29.167524,-24.902682,-907.346397,-22.537228,0.002187,0.001110
5,KernelRidge,RandomizedSearchCV,-29.579619,-1253.368321,-29.167524,-24.902682,-907.346397,-22.537228,0.002391,0.001207
6,SupportVectorRegression,GridSearchCV,-31.196103,-1368.839019,-29.596207,-0.099930,-0.009991,-0.100004,0.003191,0.001465
7,SupportVectorRegression,RandomizedSearchCV,-31.779892,-1386.793800,-30.447742,-0.099898,-0.009982,-0.099827,0.003467,0.001604
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### ARDRegression

In [98]:
regression = linear_model.ARDRegression()
parameter_grid = [{'tol':[1e-3, 1e-4], 'alpha_1':[1e-6, 1e-5, 1e-7], 'alpha_2':[1e-6, 1e-5, 1e-7]}]

In [99]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(WBC_X, WBC_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'alpha_1': 1e-07, 'alpha_2': 1e-05, 'tol': 0.0001}


In [100]:
tuned_regression = linear_model.ARDRegression(**best_parameters)
grid_scores = cross_validate(tuned_regression, WBC_X, WBC_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(WBC_df, grid_scores, 'ARDRegression', 'GridSearchCV', 8)

In [101]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(WBC_X, WBC_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'tol': 0.0001, 'alpha_2': 1e-06, 'alpha_1': 1e-07}


In [102]:
tunned_regression = linear_model.ARDRegression(**best_parameters)
random_scores = cross_validate(tunned_regression, WBC_X, WBC_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(WBC_df, random_scores, 'ARDRegression', 'RandomizedSearchCV', 9)
WBC_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,-29.727065,-1339.664090,-28.793519,-22.279819,-745.178079,-19.407585,0.001595,0.001401
1,LinearRegression,RandomizedSearchCV,-29.727065,-1339.664090,-28.793519,-22.279819,-745.178079,-19.407585,0.001397,0.000998
2,DecisionTreeRegressor,GridSearchCV,-30.358121,-1255.049823,-27.908023,-26.695304,-1007.952412,-25.005957,0.000790,0.000604
3,DecisionTreeRegressor,RandomizedSearchCV,-30.672376,-1289.762094,-27.047798,-25.208542,-919.618592,-22.573773,0.001190,0.000793
4,KernelRidge,GridSearchCV,-29.579619,-1253.368321,-29.167524,-24.902682,-907.346397,-22.537228,0.002187,0.001110
5,KernelRidge,RandomizedSearchCV,-29.579619,-1253.368321,-29.167524,-24.902682,-907.346397,-22.537228,0.002391,0.001207
6,SupportVectorRegression,GridSearchCV,-31.196103,-1368.839019,-29.596207,-0.099930,-0.009991,-0.100004,0.003191,0.001465
7,SupportVectorRegression,RandomizedSearchCV,-31.779892,-1386.793800,-30.447742,-0.099898,-0.009982,-0.099827,0.003467,0.001604
8,ARDRegression,GridSearchCV,-29.745827,-1282.080224,-27.580134,-24.105382,-853.213229,-21.831935,0.257448,0.000000
9,ARDRegression,RandomizedSearchCV,-29.745827,-1282.080226,-27.580134,-24.105382,-853.213228,-21.831935,0.304339,0.000598


In [103]:
WBC_df.iloc[:, 2:] = np.abs(WBC_df.iloc[:, 2:])
WBC_df = WBC_df.rename(columns={'test_neg_mean_absolute_error': 'test_mean_absolute_error', 'test_neg_mean_squared_error': 'test_mean_squared_error', 
                                            'test_neg_median_absolute_error': 'test_median_absolute_error', 'train_neg_mean_absolute_error': 'train_mean_absolute_error', 
                                            'train_neg_mean_squared_error': 'train_mean_squared_error', 'train_neg_median_absolute_error': 'train_median_absolute_error'})
WBC_df.head(n=10)

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,29.727065,1339.664090,28.793519,22.279819,745.178079,19.407585,0.001595,0.001401
1,LinearRegression,RandomizedSearchCV,29.727065,1339.664090,28.793519,22.279819,745.178079,19.407585,0.001397,0.000998
2,DecisionTreeRegressor,GridSearchCV,30.358121,1255.049823,27.908023,26.695304,1007.952412,25.005957,0.000790,0.000604
3,DecisionTreeRegressor,RandomizedSearchCV,30.672376,1289.762094,27.047798,25.208542,919.618592,22.573773,0.001190,0.000793
4,KernelRidge,GridSearchCV,29.579619,1253.368321,29.167524,24.902682,907.346397,22.537228,0.002187,0.001110
5,KernelRidge,RandomizedSearchCV,29.579619,1253.368321,29.167524,24.902682,907.346397,22.537228,0.002391,0.001207
6,SupportVectorRegression,GridSearchCV,31.196103,1368.839019,29.596207,0.099930,0.009991,0.100004,0.003191,0.001465
7,SupportVectorRegression,RandomizedSearchCV,31.779892,1386.793800,30.447742,0.099898,0.009982,0.099827,0.003467,0.001604
8,ARDRegression,GridSearchCV,29.745827,1282.080224,27.580134,24.105382,853.213229,21.831935,0.257448,0.000000
9,ARDRegression,RandomizedSearchCV,29.745827,1282.080226,27.580134,24.105382,853.213228,21.831935,0.304339,0.000598


In [104]:
styler = WBC_df.head(10).style.highlight_max(color='tomato').highlight_min(color='lightgreen')
styler

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,LinearRegression,GridSearchCV,29.727065,1339.664090,28.793519,22.279819,745.178079,19.407585,0.001595,0.001401
1,LinearRegression,RandomizedSearchCV,29.727065,1339.664090,28.793519,22.279819,745.178079,19.407585,0.001397,0.000998
2,DecisionTreeRegressor,GridSearchCV,30.358121,1255.049823,27.908023,26.695304,1007.952412,25.005957,0.000790,0.000604
3,DecisionTreeRegressor,RandomizedSearchCV,30.672376,1289.762094,27.047798,25.208542,919.618592,22.573773,0.001190,0.000793
4,KernelRidge,GridSearchCV,29.579619,1253.368321,29.167524,24.902682,907.346397,22.537228,0.002187,0.001110
5,KernelRidge,RandomizedSearchCV,29.579619,1253.368321,29.167524,24.902682,907.346397,22.537228,0.002391,0.001207
6,SupportVectorRegression,GridSearchCV,31.196103,1368.839019,29.596207,0.099930,0.009991,0.100004,0.003191,0.001465
7,SupportVectorRegression,RandomizedSearchCV,31.779892,1386.793800,30.447742,0.099898,0.009982,0.099827,0.003467,0.001604
8,ARDRegression,GridSearchCV,29.745827,1282.080224,27.580134,24.105382,853.213229,21.831935,0.257448,0.000000
9,ARDRegression,RandomizedSearchCV,29.745827,1282.080226,27.580134,24.105382,853.213228,21.831935,0.304339,0.000598


In [105]:
html = styler.render()
file = open("WisconsinBreastCancer_DataSet.html", "w")
file.write(html)
file.close()

## Comunities and Crime

In [213]:
zeros = np.zeros(10)
com_crime_df = pd.DataFrame({
    'Model':list(zeros), 
    'Search_strategy': list(zeros), 
    'test_neg_mean_absolute_error': list(zeros), 
    'test_neg_mean_squared_error': list(zeros), 
    'test_neg_median_absolute_error': list(zeros),
    'train_neg_mean_absolute_error': list(zeros),
    'train_neg_mean_squared_error': list(zeros),
    'train_neg_median_absolute_error': list(zeros),
    'fit_time': list(zeros),
    'score_time': list(zeros)
})
com_crime_df.head()

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [214]:
com_crime_data = pd.read_csv("data/communities.data", delimiter = ",", header = None)
com_crime_data = com_crime_data.drop(range(5), axis = 1)
com_crime_data = com_crime_data.replace("?", np.nan)
com_crime_data = com_crime_data.dropna()
com_crime_data.head()

,5,6,7,8,9,10,11,12,13,14,...,118,119,120,121,122,123,124,125,126,127
0,0.19,0.33,0.02,0.90,0.12,0.17,0.34,0.47,0.29,0.32,...,0.12,0.26,0.20,0.06,0.04,0.9,0.5,0.32,0.14,0.20
16,0.15,0.31,0.40,0.63,0.14,0.06,0.58,0.72,0.65,0.47,...,0.06,0.39,0.84,0.06,0.06,0.91,0.5,0.88,0.26,0.49
20,0.25,0.54,0.05,0.71,0.48,0.30,0.42,0.48,0.28,0.32,...,0.09,0.46,0.05,0.09,0.05,0.88,0.5,0.76,0.13,0.34
21,1.00,0.42,0.47,0.59,0.12,0.05,0.41,0.53,0.34,0.33,...,1.00,0.07,0.15,1,0.35,0.73,0,0.31,0.21,0.69
23,0.11,0.43,0.04,0.89,0.09,0.06,0.45,0.48,0.31,0.46,...,0.16,0.12,0.07,0.04,0.01,0.81,1,0.56,0.09,0.63


In [215]:
com_crime_X = com_crime_data.iloc[:, :-1].values
com_crime_y = com_crime_data.iloc[:, -1].values

### LinearRegression

In [216]:
regression = SVR(epsilon=0.01)
parameter_grid = [{'kernel': ['rbf'], 'gamma': [1e-4, 1e-3, 0.01, 0.1, 0.2, 0.5, 0.6, 0.9],'C': [1, 10, 100, 1000, 10000]}]

In [217]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(com_crime_X, com_crime_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'C': 1, 'gamma': 0.01, 'kernel': 'rbf'}


In [218]:
tuned_regression = SVR(**best_parameters)
grid_scores = cross_validate(tuned_regression, com_crime_X, com_crime_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(com_crime_df, grid_scores, 'SupportVectorRegression', 'GridSearchCV', 0)

In [219]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error')
random_search.fit(com_crime_X, com_crime_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'kernel': 'rbf', 'gamma': 0.2, 'C': 1}


In [220]:
tuned_regression = SVR(**best_parameters)
random_scores = cross_validate(tuned_regression, com_crime_X, com_crime_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(com_crime_df, random_scores, 'SupportVectorRegression', 'RandomizedSearchCV', 1)

com_crime_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,-0.129862,-0.027126,-0.101600,-0.119847,-0.023508,-0.099283,0.005180,0.002404
1,SupportVectorRegression,RandomizedSearchCV,-0.145558,-0.031550,-0.128613,-0.082505,-0.008105,-0.099830,0.008087,0.002391
2,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### LassoLars

In [221]:
regression = linear_model.LassoLars()
parameter_grid = [{'alpha':[0.02, 0.024, 0.015, 0.025, 0.026, 0.03, 0.023, 0.017, 0.033, 0.014, 0.019]}]

In [222]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(com_crime_X, com_crime_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'alpha': 0.02}


In [223]:
tuned_regression = linear_model.LassoLars(**best_parameters)
grid_scores = cross_validate(tuned_regression, com_crime_X, com_crime_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(com_crime_df, grid_scores, 'LassoLars', 'GridSearchCV', 2)

In [224]:
tuned_regression = linear_model.LassoLars(**best_parameters)
random_scores = cross_validate(tuned_regression, com_crime_X, com_crime_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(com_crime_df, random_scores, 'LassoLars', 'RandomizedSearchCV', 3)

com_crime_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,-0.129862,-0.027126,-0.101600,-0.119847,-0.023508,-0.099283,0.005180,0.002404
1,SupportVectorRegression,RandomizedSearchCV,-0.145558,-0.031550,-0.128613,-0.082505,-0.008105,-0.099830,0.008087,0.002391
2,LassoLars,GridSearchCV,-0.233331,-0.076662,-0.223616,-0.232308,-0.076076,-0.223195,0.002825,0.000958
3,LassoLars,RandomizedSearchCV,-0.233331,-0.076662,-0.223616,-0.232308,-0.076076,-0.223195,0.003174,0.000807
4,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### ARDRegression

In [225]:
regression = linear_model.ARDRegression()
parameter_grid = [{'tol':[1e-3, 1e-4], 'alpha_1':[1e-6, 1e-5, 1e-7], 'alpha_2':[1e-6, 1e-5, 1e-7]}]

In [226]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error', n_jobs = -1)
grid_search.fit(com_crime_X, com_crime_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'alpha_1': 1e-07, 'alpha_2': 1e-05, 'tol': 0.001}


In [227]:
tuned_regression = linear_model.ARDRegression(**best_parameters)
grid_scores = cross_validate(tuned_regression, com_crime_X, com_crime_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(com_crime_df, grid_scores, 'ARDRegression', 'GridSearchCV', 4)

In [228]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error', n_jobs = -1)
random_search.fit(com_crime_X, com_crime_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'tol': 0.001, 'alpha_2': 1e-05, 'alpha_1': 1e-07}


In [229]:
tuned_regression = linear_model.ARDRegression(**best_parameters)
random_scores = cross_validate(tuned_regression, com_crime_X, com_crime_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(com_crime_df, random_scores, 'ARDRegression', 'RandomizedSearchCV', 5)

com_crime_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,-0.129862,-0.027126,-0.101600,-0.119847,-0.023508,-0.099283,0.005180,0.002404
1,SupportVectorRegression,RandomizedSearchCV,-0.145558,-0.031550,-0.128613,-0.082505,-0.008105,-0.099830,0.008087,0.002391
2,LassoLars,GridSearchCV,-0.233331,-0.076662,-0.223616,-0.232308,-0.076076,-0.223195,0.002825,0.000958
3,LassoLars,RandomizedSearchCV,-0.233331,-0.076662,-0.223616,-0.232308,-0.076076,-0.223195,0.003174,0.000807
4,ARDRegression,GridSearchCV,-0.131940,-0.028597,-0.107857,-0.107035,-0.018688,-0.085781,0.670898,0.001004
5,ARDRegression,RandomizedSearchCV,-0.131940,-0.028597,-0.107857,-0.107035,-0.018688,-0.085781,0.780740,0.000997
6,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### PassiveAggressiveRegressor

In [230]:
regression = PassiveAggressiveRegressor()
parameter_grid = [{'C': np.logspace(-4, 4, 20), 'tol':[1e-3, 1e-4]}]

In [231]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error', n_jobs = -1)
grid_search.fit(com_crime_X, com_crime_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'C': 0.00026366508987303583, 'tol': 0.0001}


In [232]:
tuned_regression = PassiveAggressiveRegressor(**best_parameters)
grid_scores = cross_validate(tuned_regression, com_crime_X, com_crime_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(com_crime_df, grid_scores, 'PassiveAggressiveRegressor', 'GridSearchCV', 6)

In [233]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error', n_jobs = -1)
random_search.fit(com_crime_X, com_crime_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'tol': 0.001, 'C': 0.0018329807108324356}


In [234]:
tuned_regression = PassiveAggressiveRegressor(**best_parameters)
random_scores = cross_validate(tuned_regression, com_crime_X, com_crime_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(com_crime_df, random_scores, 'PassiveAggressiveRegressor', 'RandomizedSearchCV', 7)

com_crime_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,-0.129862,-0.027126,-0.101600,-0.119847,-0.023508,-0.099283,0.005180,0.002404
1,SupportVectorRegression,RandomizedSearchCV,-0.145558,-0.031550,-0.128613,-0.082505,-0.008105,-0.099830,0.008087,0.002391
2,LassoLars,GridSearchCV,-0.233331,-0.076662,-0.223616,-0.232308,-0.076076,-0.223195,0.002825,0.000958
3,LassoLars,RandomizedSearchCV,-0.233331,-0.076662,-0.223616,-0.232308,-0.076076,-0.223195,0.003174,0.000807
4,ARDRegression,GridSearchCV,-0.131940,-0.028597,-0.107857,-0.107035,-0.018688,-0.085781,0.670898,0.001004
5,ARDRegression,RandomizedSearchCV,-0.131940,-0.028597,-0.107857,-0.107035,-0.018688,-0.085781,0.780740,0.000997
6,PassiveAggressiveRegressor,GridSearchCV,-0.128760,-0.028020,-0.098873,-0.119860,-0.024844,-0.088000,0.007482,0.001398
7,PassiveAggressiveRegressor,RandomizedSearchCV,-0.127288,-0.027627,-0.091223,-0.117387,-0.024248,-0.084422,0.004389,0.001195
8,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


### TheilSenRegressor

In [235]:
regression = TheilSenRegressor()
parameter_grid = [{'tol':[1e-3, 1e-4], 'n_jobs':[None, 1, 2, 3, 4]}]

In [236]:
grid_search = GridSearchCV(estimator=regression, param_grid=parameter_grid, cv=5, scoring='neg_mean_squared_error', n_jobs = -1)
grid_search.fit(com_crime_X, com_crime_y)
best_parameters = grid_search.best_params_
print(best_parameters)

{'n_jobs': 3, 'tol': 0.0001}


In [237]:
tuned_regression = TheilSenRegressor(**best_parameters)
grid_scores = cross_validate(tuned_regression, com_crime_X, com_crime_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(com_crime_df, grid_scores, 'TheilSenRegressor', 'GridSearchCV', 8)

In [238]:
random_search = RandomizedSearchCV(estimator=regression, param_distributions=parameter_grid, cv=5, scoring='neg_mean_squared_error', n_jobs = -1)
random_search.fit(com_crime_X, com_crime_y)
best_parameters = random_search.best_params_
print(best_parameters)

{'tol': 0.001, 'n_jobs': None}


In [239]:
tuned_regression = TheilSenRegressor(**best_parameters)
random_scores = cross_validate(tuned_regression, com_crime_X, com_crime_y, cv=5, scoring=['neg_mean_absolute_error', 'neg_mean_squared_error', 'neg_median_absolute_error'], return_train_score=True)
populate_dataFrame(com_crime_df, random_scores, 'TheilSenRegressor', 'RandomizedSearchCV', 9)

com_crime_df.head(n=10)

,Model,Search_strategy,test_neg_mean_absolute_error,test_neg_mean_squared_error,test_neg_median_absolute_error,train_neg_mean_absolute_error,train_neg_mean_squared_error,train_neg_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,-0.129862,-0.027126,-0.101600,-0.119847,-0.023508,-0.099283,0.005180,0.002404
1,SupportVectorRegression,RandomizedSearchCV,-0.145558,-0.031550,-0.128613,-0.082505,-0.008105,-0.099830,0.008087,0.002391
2,LassoLars,GridSearchCV,-0.233331,-0.076662,-0.223616,-0.232308,-0.076076,-0.223195,0.002825,0.000958
3,LassoLars,RandomizedSearchCV,-0.233331,-0.076662,-0.223616,-0.232308,-0.076076,-0.223195,0.003174,0.000807
4,ARDRegression,GridSearchCV,-0.131940,-0.028597,-0.107857,-0.107035,-0.018688,-0.085781,0.670898,0.001004
5,ARDRegression,RandomizedSearchCV,-0.131940,-0.028597,-0.107857,-0.107035,-0.018688,-0.085781,0.780740,0.000997
6,PassiveAggressiveRegressor,GridSearchCV,-0.128760,-0.028020,-0.098873,-0.119860,-0.024844,-0.088000,0.007482,0.001398
7,PassiveAggressiveRegressor,RandomizedSearchCV,-0.127288,-0.027627,-0.091223,-0.117387,-0.024248,-0.084422,0.004389,0.001195
8,TheilSenRegressor,GridSearchCV,-0.178377,-0.059758,-0.140535,-0.096581,-0.019319,-0.071900,33.474872,0.001204
9,TheilSenRegressor,RandomizedSearchCV,-0.180589,-0.059842,-0.147755,-0.096668,-0.019313,-0.073061,148.620786,0.000998


In [240]:
com_crime_df.iloc[:, 2:] = np.abs(com_crime_df.iloc[:, 2:])
com_crime_df = com_crime_df.rename(columns={'test_neg_mean_absolute_error': 'test_mean_absolute_error', 'test_neg_mean_squared_error': 'test_mean_squared_error', 
                                            'test_neg_median_absolute_error': 'test_median_absolute_error', 'train_neg_mean_absolute_error': 'train_mean_absolute_error', 
                                            'train_neg_mean_squared_error': 'train_mean_squared_error', 'train_neg_median_absolute_error': 'train_median_absolute_error'})
com_crime_df.head(n=10)

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,0.129862,0.027126,0.101600,0.119847,0.023508,0.099283,0.005180,0.002404
1,SupportVectorRegression,RandomizedSearchCV,0.145558,0.031550,0.128613,0.082505,0.008105,0.099830,0.008087,0.002391
2,LassoLars,GridSearchCV,0.233331,0.076662,0.223616,0.232308,0.076076,0.223195,0.002825,0.000958
3,LassoLars,RandomizedSearchCV,0.233331,0.076662,0.223616,0.232308,0.076076,0.223195,0.003174,0.000807
4,ARDRegression,GridSearchCV,0.131940,0.028597,0.107857,0.107035,0.018688,0.085781,0.670898,0.001004
5,ARDRegression,RandomizedSearchCV,0.131940,0.028597,0.107857,0.107035,0.018688,0.085781,0.780740,0.000997
6,PassiveAggressiveRegressor,GridSearchCV,0.128760,0.028020,0.098873,0.119860,0.024844,0.088000,0.007482,0.001398
7,PassiveAggressiveRegressor,RandomizedSearchCV,0.127288,0.027627,0.091223,0.117387,0.024248,0.084422,0.004389,0.001195
8,TheilSenRegressor,GridSearchCV,0.178377,0.059758,0.140535,0.096581,0.019319,0.071900,33.474872,0.001204
9,TheilSenRegressor,RandomizedSearchCV,0.180589,0.059842,0.147755,0.096668,0.019313,0.073061,148.620786,0.000998


In [241]:
styler = com_crime_df.head(10).style.highlight_max(color='tomato').highlight_min(color='lightgreen')
styler

,Model,Search_strategy,test_mean_absolute_error,test_mean_squared_error,test_median_absolute_error,train_mean_absolute_error,train_mean_squared_error,train_median_absolute_error,fit_time,score_time
0,SupportVectorRegression,GridSearchCV,0.129862,0.027126,0.101600,0.119847,0.023508,0.099283,0.005180,0.002404
1,SupportVectorRegression,RandomizedSearchCV,0.145558,0.031550,0.128613,0.082505,0.008105,0.099830,0.008087,0.002391
2,LassoLars,GridSearchCV,0.233331,0.076662,0.223616,0.232308,0.076076,0.223195,0.002825,0.000958
3,LassoLars,RandomizedSearchCV,0.233331,0.076662,0.223616,0.232308,0.076076,0.223195,0.003174,0.000807
4,ARDRegression,GridSearchCV,0.131940,0.028597,0.107857,0.107035,0.018688,0.085781,0.670898,0.001004
5,ARDRegression,RandomizedSearchCV,0.131940,0.028597,0.107857,0.107035,0.018688,0.085781,0.780740,0.000997
6,PassiveAggressiveRegressor,GridSearchCV,0.128760,0.028020,0.098873,0.119860,0.024844,0.088000,0.007482,0.001398
7,PassiveAggressiveRegressor,RandomizedSearchCV,0.127288,0.027627,0.091223,0.117387,0.024248,0.084422,0.004389,0.001195
8,TheilSenRegressor,GridSearchCV,0.178377,0.059758,0.140535,0.096581,0.019319,0.071900,33.474872,0.001204
9,TheilSenRegressor,RandomizedSearchCV,0.180589,0.059842,0.147755,0.096668,0.019313,0.073061,148.620786,0.000998


In [242]:
html = styler.render()
file = open("CommunitiesCrime_DataSet.html", "w")
file.write(html)
file.close()

* SVR

Mașinile vectoriale de asistență (SVM) sunt utilizate în mod popular și pe scară largă pentru probleme de clasificare în învățarea automată.
Problema de regresie este o generalizare a problemei de clasificare, în care modelul returnează o ieșire cu valoare continuă, spre deosebire de o ieșire dintr-un set finit. Cu alte cuvinte, un model de regresie estimează o funcție multivariată cu valoare continuă.

SVM-urile rezolvă probleme de clasificare binare formulându-le ca probleme de optimizare convexă. Problema de optimizare presupune găsirea marjei maxime care separă hiperplanul, clasificând în același timp cât mai multe puncte de antrenament posibil. SVM-urile reprezintă acest hiperplan optim cu vectori de suport. Soluția rară și o generalizare bună a SVM se pretează adaptării la problemele de regresie. Generalizarea SVM la SVR se realizează prin introducerea unei regiuni insensibile la ε în jurul funcției, numită tubul ε. Acest tub reformulează problema de optimizare pentru a găsi tubul care se apropie cel mai bine de funcția cu valoare continuă, echilibrând în același timp complexitatea modelului și eroarea de predicție. Mai precis, SVR este formulat ca o problemă de optimizare prin definirea mai întâi a unei funcții de pierdere convexă ε-insensibilă care trebuie minimizată și găsirea celui mai plat tub care conține majoritatea instanțelor de antrenament. Prin urmare, o funcție multiobjectivă este construită din funcția de pierdere și proprietățile geometrice ale tubului. Apoi, optimizarea convexă, care are o soluție unică, este rezolvată, utilizând algoritmi de optimizare numerici corespunzători. Hiperplanul este reprezentat în termeni de vectori de susținere, care sunt probe de antrenament care se află în afara limitei tubului. La fel ca în SVM, vectorii de sprijin din SVR sunt cele mai influente instanțe care afectează forma tubului, iar datele de antrenament și de testare sunt presupuse a fi independente și distribuite identic (iid), extrase din aceeași funcție de distribuție de probabilitate fixă, dar necunoscută. într-un context de învățare supravegheată.

Formularea problemei SVR este adesea derivată cel mai bine dintr-o perspectivă geometrică, folosind exemplul unidimensional din imaginea de mai jos.

<img src="./images/SVM_1.png" alt="drawing" width="600px"/>

Ceea ce încercăm să facem aici este practic să încercăm să decidem o limită de decizie la o distanță „e” de hiperplanul original, astfel încât punctele de date cele mai apropiate de hiperplanul sau vectorii de susținere să fie în acea linie de limită

<img src="./images/SVM_2.jpeg" alt="drawing" width="600px"/>

https://link.springer.com/chapter/10.1007/978-1-4302-5990-9_4

https://www.analyticsvidhya.com/blog/2020/03/support-vector-regression-tutorial-for-machine-learning/

* LassoLars

În statistici, regresia cu unghi minim (LARS) este un algoritm pentru adaptarea modelelor de regresie liniară la date de dimensiuni ridicate, dezvoltat de Bradley Efron, Trevor Hastie, Iain Johnstone și Robert Tibshirani.

Să presupunem că ne așteptăm ca o variabilă de răspuns să fie determinată de o combinație liniară a unui subset de covariabile potențiale. Apoi, algoritmul LARS oferă un mijloc de a produce o estimare a variabilelor de inclus, precum și a coeficienților acestora.

În loc să dea un rezultat vector, soluția LARS constă dintr-o curbă care denotă soluția pentru fiecare valoare a normei L1 a vectorului parametru. Algoritmul este similar cu regresia pas cu pas înainte, dar în loc să includă variabile la fiecare pas, parametrii estimate sunt crescuți într-o direcție echiangulară cu corelațiile fiecăruia cu reziduul.

O explicație simplă a regresiei Lasso și Least Angle


Dați un set de măsurători de intrare x1, x2 ... xp și o măsurare a rezultatului y, lazo se potrivește unui model liniar

yhat = b0 + b1 * x1 + b2 * x2 + ... bp * xp

Criteriul pe care îl folosește este:

Minimizați suma ((y-yhat) ^ 2) sub rezerva sumei [valoare absolută (bj)] <= s

Prima sumă este preluată de observațiile (cazurile) din setul de date. „S” legat este un parametru de reglare. Când „s” este suficient de mare, constrângerea nu are efect și soluția este doar regresia obișnuită multiplă liniară obișnuită a lui y pe x1, x2, ... xp.

Cu toate acestea, atunci când pentru valori mai mici de s (s> = 0) soluțiile sunt versiuni micșorate ale estimărilor celor mai mici pătrate. Adesea, unii dintre coeficienții bj sunt zero. Alegerea „s” este ca alegerea numărului de predictori de utilizat într-un model de regresie, iar validarea încrucișată este un instrument bun pentru estimarea celei mai bune valori pentru „s”.


* Calculul soluțiilor Lasso

Calculul soluțiilor lazo este o problemă de programare pătratică și poate fi abordat prin algoritmi standard de analiză numerică. Dar procedura de regresie cu unghi minim este o abordare mai bună. Acest algoritm exploatează structura specială a problemei lazo și oferă o modalitate eficientă de a calcula soluțiile simultan pentru toate valorile „s”.

Regresia cu unghi minim este ca o versiune mai „democratică” a regresiei în trepte. Amintiți-vă cum funcționează regresia în trepte:

* Algoritmul de regresie pas cu pas înainte:

Începeți cu toți coeficienții bj egali cu zero.
Găsiți predictorul xj cel mai corelat cu y și adăugați-l în model. Luați reziduuri r = y-yhat.
Continuați, în fiecare etapă adăugând la model predictorul cel mai corelat cu r.
Până la: toți predictorii sunt în model
Cea mai mică procedură de regresie a unghiului urmează aceeași schemă generală, dar nu adaugă un predictor pe deplin în model. Coeficientul acelui predictor este crescut numai până când acel predictor nu mai este cel mai corelat cu r rezidual. Apoi, un alt predictor concurent este invitat să „se alăture clubului”.


* Algoritmul de regresie cu unghi minim:

Începeți cu toți coeficienții bj egali cu zero.
Găsiți predictorul xj cel mai corelat cu y
Creșteți coeficientul bj în direcția semnului corelației sale cu y. Luați reziduuri r = y-yhat pe parcurs. Opriți-vă când un alt predictor xk are la fel de multă corelație cu r ca xj.
Creșteți (bj, bk) în direcția articulației lor minime pătrate, până când un alt predictor xm are la fel de multă corelație cu r rezidual.
Continuați până: toți predictorii sunt în model
În mod surprinzător, se poate demonstra că, cu o modificare, această procedură oferă întreaga cale a soluțiilor lazo, deoarece s variază de la 0 la infinit. Modificarea necesară este: dacă un coeficient diferit de zero atinge zero, eliminați-l din setul activ de predictori și recomputați direcția articulației.

<img src="./images/LassoLars.png" alt="drawing" width="600px"/>

https://statweb.stanford.edu/~tibs/lasso/simple.html

https://en.wikipedia.org/wiki/Least-angle_regression

ARD

Determinarea automată a relevanței (ARD) se bazează pe metoda inferenței bayesiene. Scikit-learn API oferă clasa ARDRegression pentru a se potrivi modelului de regresie utilizând metoda ARD. ARDRegresiunea consideră ponderile modelului ca un Gaussian distribuite și estimează parametrii lambda și alfa prin iterație.

Un sistem de regresie de determinare automată a relevanței (in engleza Automatic Relevance Determination (ARD)) este un sistem de regresie bayesiană care implementează un algoritm de regresie ARD pentru a rezolva o sarcină de regresie ARD.

Pentru a motiva Determinarea Automată a Relevanței (ARD) se stabilește o intuiție pentru problema alegerii unui model complex care se potrivește bine cu datele comparativ cu un model simplu care generalizează bine. Prin urmare, ideea din spatele aparatului de ras este prezentată ca o modalitate de echilibrare a prejudecății și varianței. Acest lucru ne conduce la cadrul matematic al interpolației Bayesiene și la selectarea modelelor pentru a alege între diferite modele bazate pe date.

Pentru a obține ARD cât mai rapid posibil, se repetă elementele de bază matematice ale unui model liniar simplu, precum și ideea de regularizare pentru a preveni supraadaptarea. Pe baza acestui fapt, se introduce Regresia Bayesian Ridge (BayesianRidge în Scikit-Learn). Generalizarea conceptului de regresie Bayesian Ridge ne duce și mai mult la ideea din spatele ARD (ARDRegresie în Scikit-Learn).

Cu ajutorul unui exemplu practic, consolidăm ceea ce s-a învățat până acum și comparăm ARD cu un model obișnuit de cel puțin pătrat. Acum ne adâncim în matematica ARD și prezentăm algoritmul care rezolvă problema de minimizare a ARD. În cele din urmă, sunt discutate câteva detalii despre implementarea ARD a Scikit-Learn.

https://florianwilhelm.info/2016/03/explaining_the_idea_behind_ard/

http://www.gabormelli.com/RKB/Automatic_Relevance_Determination_(ARD)_Regression_System

<img src="./images/ARD.png" alt="drawing" width="600px"/>

* Passive Aggresive Regressor

Algoritmii pasiv-agresivi sunt utilizați în general pentru învățarea pe scară largă. Este unul dintre puținii „algoritmi de învățare online”. În algoritmii de învățare automată online, datele de intrare vin în ordine secvențială, iar modelul de învățare automată este actualizat pas cu pas, spre deosebire de învățarea în serie, unde întregul set de date de instruire este utilizat simultan. Acest lucru este foarte util în situațiile în care există o cantitate uriașă de date și este calculabil imposibil să instruiți întregul set de date din cauza dimensiunii foarte mari a datelor. Putem spune pur și simplu că un algoritm de învățare online va obține un exemplu de instruire, va actualiza clasificatorul și apoi va arunca exemplul.
Un exemplu foarte bun în acest sens ar fi detectarea știrilor false pe un site de socializare precum Twitter, unde se adaugă date noi în fiecare secundă. Pentru a citi în mod dinamic datele de pe Twitter în mod continuu, datele ar fi uriașe și ar fi ideală utilizarea unui algoritm de învățare online.

Algoritmii pasiv-agresivi sunt oarecum similari cu un model Perceptron, în sensul că nu necesită o rată de învățare. Cu toate acestea, acestea includ un parametru de regularizare.

Cum funcționează algoritmii pasiv-agresivi:
Algoritmii pasiv-agresivi sunt numiți astfel deoarece:



Pasiv: dacă predicția este corectă, păstrați modelul și nu efectuați modificări. adică datele din exemplu nu sunt suficiente pentru a provoca modificări ale modelului.
Agresiv: dacă predicția este incorectă, modificați modelul. adică, unele modificări ale modelului îl pot corecta.
Înțelegerea matematicii din spatele acestui algoritm nu este foarte simplă și depășește scopul unui singur articol. Acest articol oferă doar o prezentare generală a algoritmului și o simplă implementare a acestuia. Pentru a afla mai multe despre matematica din spatele acestui algoritm, vă recomand să urmăriți acest videoclip excelent despre lucrările algoritmului de către Dr. Victor Lavrenko.

Parametri importanți:

C: Acesta este parametrul de regularizare și denotă penalizarea pe care o va face modelul pentru o predicție incorectă
max_iter: numărul maxim de iterații pe care modelul le face asupra datelor de instruire.
tol: Criteriul de oprire. Dacă este setat la None, modelul se va opri când (pierdere> previous_loss - tol). În mod implicit, este setat la 1e-3.

<img src="./images/PAR.png" alt="drawing" width="600px"/>

https://www.bonaccorso.eu/2017/10/06/ml-algorithms-addendum-passive-aggressive-algorithms/

https://www.geeksforgeeks.org/passive-aggressive-classifiers/

* TheilSenRegressor

În statisticile neparametrice, estimatorul Theil – Sen este o metodă de potrivire robustă a unei linii la eșantionarea punctelor în plan (regresie liniară simplă) prin alegerea medianei pantei tuturor liniilor prin perechi de puncte. De asemenea, a fost numit estimatorul de pantă al lui Sen, selecția pantei, metoda mediană unică, metoda Kendall robustă de potrivire a liniei, și linia robustă Kendall-Theil .  Acesta poartă numele lui Henri Theil și Pranab K. Sen, care au publicat articole despre această metodă în 1950 și respectiv în 1968  și după Maurice Kendall datorită relației sale cu coeficientul de corelație a rangului tau Kendall.

Acest estimator poate fi calculat eficient și este insensibil la valori anormale. Poate fi semnificativ mai precis decât regresia liniară simplă non-robustă (cele mai mici pătrate) pentru datele înclinate și heteroskedastice și concurează bine împotriva celor mai mici pătrate chiar și pentru datele distribuite în mod normal în termeni de putere statistică. A fost numită „cea mai populară tehnică neparametrică pentru estimarea unei tendințe liniare”.

Panta mediană a unui set de n puncte de eșantionare poate fi calculată exact calculând toate O (n2) drepte prin perechi de puncte și apoi aplicând un algoritm liniar de găsire a medianei timpului. Alternativ, poate fi estimată prin eșantionarea perechilor de puncte. Această problemă este echivalentă, în cadrul dualității proiective, cu problema găsirii punctului de trecere într-un aranjament de linii care are coordonata x mediană între toate aceste puncte de trecere.

Problema efectuării selecției pantei exact, dar mai eficient decât algoritmul timpului pătratic al forței brute a fost studiată pe larg în geometria de calcul. Sunt cunoscute mai multe metode diferite pentru calcularea estimatorului Theil – Sen exact în timpul O (n log n), fie deterministic , fie folosind algoritmi randomizați. Estimatorul mediu repetat al lui Siegel poate fi, de asemenea, construit în același timp legat. În modelele de calcul în care coordonatele de intrare sunt numere întregi și în care operațiunile bit-bit pe numere întregi necesită timp constant, estimatorul Theil-Sen poate fi construit și mai rapid, în timp aleatorizat așteptat {\ displaystyle O (n {\ sqrt {\ log n}})} O (n {\ sqrt {\ log n}}).

Un estimator pentru panta cu rang aproximativ mediu, având același punct de defalcare ca și estimatorul Theil-Sen, poate fi menținut în modelul fluxului de date (în care punctele de eșantionare sunt procesate unul câte unul de un algoritm care nu are suficientă persistență stocare pentru a reprezenta întregul set de date) folosind un algoritm bazat pe rețele ε.

În pachetul de statistici R, atât estimatorul Theil – Sen, cât și estimatorul mediu repetat al lui Siegel sunt disponibile prin biblioteca mblm. O aplicație Visual Basic independentă pentru estimarea Theil-Sen, KTRLine, a fost pusă la dispoziție de US Geological Survey. Estimatorul Theil – Sen a fost implementat și în Python ca parte a bibliotecilor SciPy și scikit-learn.

https://en.wikipedia.org/wiki/Theil%E2%80%93Sen_estimator

http://home.olemiss.edu/~xdang/papers/MTSE.pdf

<img src="./images/TSE.jpg" alt="drawing" width="600px"/>